In [1]:
import json
from nltk.tokenize import TweetTokenizer
import copy


In [2]:
from qclassify import qclass
classifier = qclass()


In [3]:
def split_que_ans(game,human=False):
    new_game = copy.deepcopy(game)
    for key in game:
        new_game[key] = copy.deepcopy(game[key])
        if human:
            gen_dialogue = [game[key]['true_dialogue']]
            gen_dialogue[0] = gen_dialogue[0].replace('Yes','<Yes>')
            gen_dialogue[0] = gen_dialogue[0].replace('yes','<yes>')
            gen_dialogue[0] = gen_dialogue[0].replace('No','<No>')
            gen_dialogue[0] = gen_dialogue[0].replace('no','<no>')
            gen_dialogue[0] = gen_dialogue[0].replace('NA','<NA>')
            gen_dialogue[0] = gen_dialogue[0].replace('na','<na>')
            new_game[key]['gen_dialogue'] = copy.deepcopy(game[key]['true_dialogue'])
        else:
            gen_dialogue = [game[key]['gen_dialogue']] 
        gen_dialogue[0] = gen_dialogue[0].replace('<start>','')
        
        # replace < and > in unk token so we can split on that one next
        gen_dialogue[0] = gen_dialogue[0].replace('<unk>', '_unk_')
        
        tmp_gen_dialogue = [dial.split('>') for dial in gen_dialogue]
        
        new_game[key]['que'] = []
        new_game[key]['ans'] = []
        for dialogue in tmp_gen_dialogue[0]:
            dialogue = dialogue.replace('<','')
            if dialogue:
                try:
                    que,ans = dialogue.split('?')
                    que = que + ' ?'
                    new_game[key]['que'].append(que)
                    new_game[key]['ans'].append(ans.lower().strip())
                except:
                    nothing = 1
    print('Spliting is done')                
    return(new_game)

In [4]:
def question_type_shift_yes_answer(game,cond='all'):
    # category : 'super-category', 'object'
    # cond: 'all' for all game, 'success' for successful game and 'un-success' for un-successful game

    out = {}
    for category in ['<super-category>', '<object>']:
        all_categories = ['<super-category>', '<object>', '<attribute>']
        count = 0
        cond_count = 0
        e_count =0

        if category == '<super-category>':
            non_category = ['<object>', '<attribute>']
        elif category == '<object>':
            non_category = ['<attribute>']
        else:
            print('check category')

        l = 0
        for key in game:
            l+=1
            if cond == 'success':
                if game[key]['target_id'] != game[key]['guess_id']:
                    continue
            elif cond == 'un-success': 
                if game[key]['target_id'] == game[key]['guess_id']:
                    continue

            q_ann = game[key]['q_ann_at']   

            ans = game[key]['ans']
            indices = [i for i, x in enumerate(q_ann) if x == category]
            for i_idx in range(0,len(indices)):
                if i_idx != len(indices):
                    try:
                        next_q_ann = q_ann[indices[i_idx]+1]
                    except:
                        e_count +=1
                        continue
                    curr_q_ans = ans[indices[i_idx]].lower()

                    if curr_q_ans == 'yes':
                        count +=1
                        if next_q_ann in non_category:
                            cond_count += 1

    #     print(cond_count*100/count,count,cond_count,e_count,cond_count*100/(count+e_count))
        
        out[category] = cond_count*100/count 
    return out

In [5]:
def get_que_type_multi(game,q_max_miss=0,file_name='dummy.json'):
    
    # Classification of game
    
    # ATTRIBUTES:
        # - color
        # - shape
        # - size
        # - texture
        # - action
        # - spatial/location
       
    # ENTITY
        # - super-category
        # - object

    tknzr = TweetTokenizer(preserve_case=False)
    q_count = 0
    
    d_count =0
    number_count = 0
    color_count = 0
    shape_count = 0
    size_count = 0
    texture_count = 0
    action_count = 0
    spatial_count = 0
    object_count = 0
    super_count = 0
    
    att_count = 0
    
    new_game = copy.deepcopy(game)
    
    for key in game:
        new_game[key]['q_ann'] = [] #To store categoty
        new_game[key]['q_ann_at'] = [] #To store broad categoty
        
        d_count +=1
        d_flag = True
        d_flag_count = 0
        questions = game[key]['que']
        game[key]['gen_dialogue'] = game[key]['gen_dialogue'].replace('?', ' ?')
        dial =  [n for x, n in enumerate(game[key]['gen_dialogue'].split())]
    
        for que_idx, que in enumerate(questions):
            q_count +=1
            cat = '<NA>'
            que = que.lower()
            cat = classifier.que_classify_multi(que)
            
            att_flag = False
            
            if '<color>' in cat:
                color_count +=1
                att_flag = True
                
            if '<shape>' in cat:
                shape_count +=1
                att_flag = True
            if '<size>' in cat:
                size_count +=1
                att_flag = True
            if '<texture>' in cat:
                texture_count +=1
                att_flag = True
            if '<action>' in cat:
                action_count +=1
                att_flag = True
            if '<spatial>' in cat:
                spatial_count +=1
                att_flag = True
            if att_flag:
                att_count +=1
                
            if '<object>' in cat:
                object_count +=1
            if '<super-category>' in cat:
                super_count +=1
                
            new_game[key]['q_ann'].append(cat)
            if cat == '<NA>' or cat == '<object>' or cat == '<super-category>':
                new_game[key]['q_ann_at'].append(cat)
            else:
                new_game[key]['q_ann_at'].append("<attribute>")
#             break
            
    
    ent_count = object_count + super_count
    
    ent_per = ent_count * 100 / q_count
    object_per = object_count * 100 / q_count
    super_per = super_count * 100 / q_count
    
    att_per = att_count * 100 / q_count
    color_per = color_count * 100 / q_count
    shape_per = shape_count * 100 / q_count
    size_per = size_count * 100 / q_count
    texture_per = texture_count * 100 / q_count
    spatial_per = spatial_count * 100 / q_count
    action_per = action_count * 100 / q_count
    rest_per = 100 - (att_per+ent_per) 
    
#     print('Entity : ', ent_per)
#     print('Super : ', super_per)
#     print('Object : ', object_per)
#     print('Attribute : ', att_per)
#     print('Color : ', color_per)
#     print('Shape : ', shape_per)
#     print('Size : ', size_per)
#     print('Texture : ', texture_per)
#     print('Location : ', spatial_per)
#     print('Action : ', action_per)
#     print('NA : ', rest_per)
    
    out = {}
    out['Entity']= ent_per
    out['Super ']=  super_per
    out['Object']= object_per
    out['Attribute']= att_per
    out['Color']= color_per
    out['Shape']= shape_per
    out['Size']= size_per
    out['Texture']= texture_per
    out['Location']= spatial_per
    out['Action']= action_per
    out['NA']= rest_per
    print(out)
    return out, new_game

In [6]:
file_name = '../../data/dummy.json'

In [7]:
with open(file_name) as file:
    game = json.load(file)

In [8]:
game = split_que_ans(game)

Spliting is done


In [9]:
#TABLE 3 in the NAACL Paper
stat, new_game = get_que_type_multi(game)

{'Entity': 28.533528420008157, 'Super ': 9.336831439226717, 'Object': 19.19669698078144, 'Attribute': 70.13071648104001, 'Color': 17.733126474186754, 'Shape': 0.02690850687218039, 'Size': 0.7139163229525359, 'Texture': 0.08535042023519716, 'Location': 66.77177802163612, 'Action': 3.1524997582438834, 'NA': 1.3357550989518359}


In [10]:
#TABLE 4 in the NAACL Paper
shift_cat = question_type_shift_yes_answer(new_game)
print(shift_cat)


{'<super-category>': 98.38142902779256, '<object>': 95.98132552178689}
